In [2]:
import json
import pandas as pd
import re

def parse_log_file(log_file_path):
    with open(log_file_path, 'r', encoding='utf-8') as file:
        log_data = file.read()
    
    # Find indices of sections
    sandbox_index = log_data.find("Sandbox logs:")
    activities_index = log_data.find("Activities log:")
    trade_index = log_data.find("Trade History:")
    
    # Extract raw strings
    sandbox_logs_raw = log_data[sandbox_index + len("Sandbox logs:"):activities_index].strip()
    activities_log_raw = log_data[activities_index + len("Activities log:"):trade_index].strip()
    trade_history_raw = log_data[trade_index + len("Trade History:"):].strip()
    
    # Parse Sandbox logs (JSON objects)
    sandbox_logs = [json.loads(obj) for obj in re.findall(r'\{.*?\}', sandbox_logs_raw, re.DOTALL)]
    
    # Parse Trade History (JSON list)
    trade_history = json.loads(trade_history_raw) if trade_history_raw.startswith('[') else []
    
    # Read Activities log as CSV
    from io import StringIO
    activities_df = pd.read_csv(StringIO(activities_log_raw), delimiter=';')
    
    return sandbox_logs, activities_df, trade_history

# Example usage
log_file_path = "logs/empty_submission.log"  # Change this to the actual log file path
sandbox_logs, market_data, trade_history = parse_log_file(log_file_path)

In [3]:
resin = market_data[market_data['product'] == 'RAINFOREST_RESIN']
kelp = market_data[market_data['product'] == 'KELP']

In [4]:
resin['timestamp'].value_counts().value_counts()

count
1    2000
Name: count, dtype: int64

In [5]:
import plotly.express as px

fig = px.line(resin, x='timestamp', y='mid_price', title='RAINFOREST_RESIN price over time')
fig.show()

In [6]:
fig = px.line(kelp, x='timestamp', y='mid_price', title='KELP price over time')
fig.show()